# Analisis Base de datos

In [29]:
import os
import pandas as pd
import numpy as np

## Open saved DB

In [2]:
os.chdir('./../data/04 Wekun')
users = pd.read_csv('users_16-09-2019.csv')
answers = pd.read_csv('answers_16-09-2019.csv')

In [3]:
users.columns

Index(['id', 'user_id', 'device_id', 'create_at', 'age', 'gender', 'education',
       'transport', 'nationality', 'ip_address', 'comuna', 'region', 'country',
       'browser', 'platform', 'psico1', 'psico2', 'psico3', 'psico4', 'psico5',
       'psico6', 'psico7', 'psico8', 'psico9', 'psico10', 'income'],
      dtype='object')

In [4]:
answers['choice'] = answers['choice'].replace({'image_1':1,
                                   'image_2':2,
                                   '=':3,
                                   '= ':3,
                                   'X':99})
answers['choice'] = answers['choice'].astype(int)

In [5]:
users.rename(columns={'id':'user'}, inplace=True)

       
users['gender'] = users['gender'].replace({'M':0,
                                   'F':1,
                                  'O':99})
users['education'] =  users['education'].replace({'es1':1,'en1':1,
                                          'es2i':1,'en2i':1,
                                          'es3i':2,'en3i':2,
                                          'es4i':3,'en4i':3,
                                          'es5i':3,'en5i':3,
                                          'es6i':5,'en6i':5,
                                          'es2':2,'en2':2,
                                          'es3':3,'en3':3,
                                          'es4':4,'en4':4,
                                          'es5':5,'en5':5,
                                          'es6':6,'en6':6})
users = users.astype({'user_id':str,
              'user':int,
              'age':float,
              'gender':int,
              'education':int,
              'transport':int})
age_outlayers = users[(users["age"]>= 100) | \
                     (users["age"]< 0)]

users = users[(users["age"]< 100) & \
         (users["age"]>= 0)]

users = users.astype({'age':int})

In [6]:
df = pd.merge(answers,users, how='inner', on='user_id', sort=False, indicator=False,suffixes=('_ans', '_usr'))

In [7]:
df.head()

,id,user_id,img_1,img_2,question,choice,create_at_ans,user,device_id,create_at_usr,...,psico2,psico3,psico4,psico5,psico6,psico7,psico8,psico9,psico10,income
0,0,6211ef85-5708-11e9-b416-005056b5408a,513da00ffdc9f035870088ac,5140d7c5fdc9f0492600389c,3,99,2019-04-04 18:35:16.274433,1,0ed3319d-5231-11e9-b416-005056b5408a,2019-04-04 18:35:12.852640,...,++,0,++,--,+,++,++,-,0,0
1,1,6211ef85-5708-11e9-b416-005056b5408a,50f5eb69fdc9f065f00082ba,50f5ed99fdc9f065f0008e24,3,1,2019-04-09 17:01:19.859871,1,0ed3319d-5231-11e9-b416-005056b5408a,2019-04-04 18:35:12.852640,...,++,0,++,--,+,++,++,-,0,0
2,3,6211ef85-5708-11e9-b416-005056b5408a,51413469fdc9f0492600464d,513da0effdc9f03587008a5d,3,1,2019-04-04 18:35:42.310486,1,0ed3319d-5231-11e9-b416-005056b5408a,2019-04-04 18:35:12.852640,...,++,0,++,--,+,++,++,-,0,0
3,7,6211ef85-5708-11e9-b416-005056b5408a,50f5e701fdc9f065f0006f0b,50f563cbfdc9f065f0005e3e,3,2,2019-04-04 18:35:51.231727,1,0ed3319d-5231-11e9-b416-005056b5408a,2019-04-04 18:35:12.852640,...,++,0,++,--,+,++,++,-,0,0
4,10,6211ef85-5708-11e9-b416-005056b5408a,51409c81fdc9f049260011df,513d9d31fdc9f03587007fb5,3,2,2019-04-04 18:36:30.939858,1,0ed3319d-5231-11e9-b416-005056b5408a,2019-04-04 18:35:12.852640,...,++,0,++,--,+,++,++,-,0,0


In [8]:
df.shape

(27275, 32)

## Place Pulse

In [15]:
# PP = pd.read_csv('./../05 Benchmark/PP2_votes.csv')

In [41]:
# PP['winner'] = PP['winner'].replace({'left':1,
#                                     'right':2,
#                                     'equal':3})

In [42]:
# PP.head()

In [43]:
# models_db = PP[['left_id', 'right_id','winner', 'category']]
# models_db.columns = pd.Index(['img_1', 'img_2', 'choice','question'])

# Generar archivo .DAT

In [9]:
os.chdir('./../01 PP')

In [10]:
features = pd.read_csv('all_features_imagenes.csv')
models_db = df

In [11]:
models_db = models_db.merge(features, how='inner', left_on='img_1', right_on='ID', sort=False, suffixes=('', '_izq'))
models_db = models_db.merge(features, how='inner', left_on='img_2', right_on='ID', sort=False, suffixes=('_izq', '_der'))
models_db = models_db.drop(columns=['img_1','img_2','ID_izq','ID_der'])

In [12]:
models_db.shape

(27275, 136)

In [13]:
outliers = models_db[(models_db['NA_izq'] != 0) | (models_db['NA_der'] != 0)]
#                      | \
#              (models_db['outlier_bn_izq'] != 0) | (models_db['outlier_bn_der'] != 0) | \
#              (models_db['outlier_hs_izq'] != 0) | (models_db['outlier_hs_der'] != 0) | \
#              (models_db['outlier_hlf_izq'] > 1) | (models_db['outlier_hlf_der'] > 1)]

models_db = models_db[(models_db['NA_izq'] == 0) & (models_db['NA_der'] == 0)]
#                       & \
#              (models_db['outlier_bn_izq'] == 0) & (models_db['outlier_bn_der'] == 0) & \
#              (models_db['outlier_hs_izq'] == 0) & (models_db['outlier_hs_der'] == 0) & \
#              (models_db['outlier_hlf_izq'] <= 1) & (models_db['outlier_hlf_der'] <= 1)]

In [14]:
models_db.shape[0] + outliers.shape[0]

27275

In [15]:
outliers.shape

(362, 136)

In [16]:
# models_db.head()
models_db.shape

(26913, 136)

In [17]:
models_db.iloc[:,:40].columns

Index(['id', 'user_id', 'question', 'choice', 'create_at_ans', 'user',
       'device_id', 'create_at_usr', 'age', 'gender', 'education', 'transport',
       'nationality', 'ip_address', 'comuna', 'region', 'country', 'browser',
       'platform', 'psico1', 'psico2', 'psico3', 'psico4', 'psico5', 'psico6',
       'psico7', 'psico8', 'psico9', 'psico10', 'income', 'Unnamed: 0_izq',
       'NA_izq', 'blobs_izq', 'edges_izq', 'mean_bn_izq', 'mean_bs_izq',
       'mean_h_izq', 'mean_l_izq', 'mean_s_izq', 'stdev_bn_izq'],
      dtype='object')

In [18]:
outliers.loc[:,['ID_izq','ID_der',
                'NA_izq','NA_der',
                'outlier_bn_izq','outlier_bn_der',
                'outlier_hs_izq','outlier_hs_der',
                'outlier_hlf_izq','outlier_hlf_der']].head()

KeyError: "['ID_izq', 'ID_der', 'outlier_bn_izq', 'outlier_bn_der', 'outlier_hs_izq', 'outlier_hs_der', 'outlier_hlf_izq', 'outlier_hlf_der'] not in index"

In [19]:
users = models_db[['question', 'choice', 'user', 'age', 'gender', 'education', 'transport']]
# users = models_db[['question', 'choice']]
users.head()

,question,choice,user,age,gender,education,transport
1,3,1,1,24,0,5,3
2,3,1,1,24,0,5,3
3,2,1,821,38,1,5,3
4,3,2,1,24,0,5,3
5,3,2,1,24,0,5,3


In [20]:
data = pd.DataFrame()

# data['N_Bicyclist'] = models_db['Bicyclist_N_izq'] - models_db['Bicyclist_N_der']
# data['S_Bicyclist'] = models_db['Bicyclist_S_izq'] - models_db['Bicyclist_S_der']
# data['N_Building'] = models_db['Building_N_izq'] - models_db['Building_N_der']
# data['S_Building'] = models_db['Building_S_izq'] - models_db['Building_S_der']
# data['N_Car'] = models_db['Car_N_izq'] - models_db['Car_N_der']
# data['S_Car'] = models_db['Car_S_izq'] - models_db['Car_S_der']
# data['N_Fence'] = models_db['Fence_N_izq'] - models_db['Fence_N_der']
# data['S_Fence'] = models_db['Fence_S_izq'] - models_db['Fence_S_der']
# data['N_Pavement'] = models_db['Pavement_N_izq'] - models_db['Pavement_N_der']
# data['S_Pavement'] = models_db['Pavement_S_izq'] - models_db['Pavement_S_der']
# data['N_Pedestrian'] = models_db['Pedestrian_N_izq'] - models_db['Pedestrian_N_der']
# data['S_Pedestrian'] = models_db['Pedestrian_S_izq'] - models_db['Pedestrian_S_der']
# data['N_Pole'] = models_db['Pole_N_izq'] - models_db['Pole_N_der']
# data['S_Pole'] = models_db['Pole_S_izq'] - models_db['Pole_S_der']
# data['N_Road'] = models_db['Road_N_izq'] - models_db['Road_N_der']
# data['S_Road'] = models_db['Road_S_izq'] - models_db['Road_S_der']
# data['N_SignSymbol'] = models_db['SignSymbol_N_izq'] - models_db['SignSymbol_N_der']
# data['S_SignSymbol'] = models_db['SignSymbol_S_izq'] - models_db['SignSymbol_S_der']
# data['N_Sky'] = models_db['Sky_N_izq'] - models_db['Sky_N_der']
# data['S_Sky'] = models_db['Sky_S_izq'] - models_db['Sky_S_der']
# data['N_Tree'] = models_db['Tree_N_izq'] - models_db['Tree_N_der']
# data['S_Tree'] = models_db['Tree_S_izq'] - models_db['Tree_S_der']
data['N_Bicyclist'] = 0.1 * np.log(models_db['Bicyclist_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Bicyclist_N_izq'] * 100 + 1)
data['S_Bicyclist'] = 0.1 * np.log(models_db['Bicyclist_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Bicyclist_S_izq'] * 100 + 1)
data['N_Building'] = 0.1 * np.log(models_db['Building_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Building_N_izq'] * 100 + 1)
data['S_Building'] = 0.1 * np.log(models_db['Building_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Building_S_izq'] * 100 + 1)
data['N_Car'] = 0.1 * np.log(models_db['Car_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Car_N_izq'] * 100 + 1)
data['S_Car'] = 0.1 * np.log(models_db['Car_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Car_S_izq'] * 100 + 1)
data['N_Fence'] = 0.1 * np.log(models_db['Fence_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Fence_N_izq'] * 100 + 1)
data['S_Fence'] = 0.1 * np.log(models_db['Fence_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Fence_S_izq'] * 100 + 1)
data['N_Pavement'] = 0.1 * np.log(models_db['Pavement_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Pavement_N_izq'] * 100 + 1)
data['S_Pavement'] = 0.1 * np.log(models_db['Pavement_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Pavement_S_izq'] * 100 + 1)
data['N_Pedestrian'] = 0.1 * np.log(models_db['Pedestrian_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Pedestrian_N_izq'] * 100 + 1)
data['S_Pedestrian'] = 0.1 * np.log(models_db['Pedestrian_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Pedestrian_S_izq'] * 100 + 1)
data['N_Pole'] = 0.1 * np.log(models_db['Pole_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Pole_N_izq'] * 100 + 1)
data['S_Pole'] = 0.1 * np.log(models_db['Pole_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Pole_S_izq'] * 100 + 1)
data['N_Road'] = 0.1 * np.log(models_db['Road_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Road_N_izq'] * 100 + 1)
data['S_Road'] = 0.1 * np.log(models_db['Road_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Road_S_izq'] * 100 + 1)
data['N_SignSymbol'] = 0.1 * np.log(models_db['SignSymbol_N_der'] * 100 + 1) - 0.1 * np.log(models_db['SignSymbol_N_izq'] * 100 + 1)
data['S_SignSymbol'] = 0.1 * np.log(models_db['SignSymbol_S_der'] * 100 + 1) - 0.1 * np.log(models_db['SignSymbol_S_izq'] * 100 + 1)
data['N_Sky'] = 0.1 * np.log(models_db['Sky_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Sky_N_izq'] * 100 + 1)
data['S_Sky'] = 0.1 * np.log(models_db['Sky_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Sky_S_izq'] * 100 + 1)
data['N_Tree'] = 0.1 * np.log(models_db['Tree_N_der'] * 100 + 1) - 0.1 * np.log(models_db['Tree_N_izq'] * 100 + 1)
data['S_Tree'] = 0.1 * np.log(models_db['Tree_S_der'] * 100 + 1) - 0.1 * np.log(models_db['Tree_S_izq'] * 100 + 1)

# data = (data-data.mean())/data.std()
# data = data*10

data['blobs'] = models_db['blobs_der'] - models_db['blobs_izq']
data['edges'] = models_db['edges_der'] - models_db['edges_izq']
data['mean_bn'] = models_db['mean_bn_der'] - models_db['mean_bn_izq']
data['mean_h'] = models_db['mean_h_der'] - models_db['mean_h_izq']
data['mean_l'] = models_db['mean_l_der'] - models_db['mean_l_izq']
data['mean_s'] = models_db['mean_s_der'] - models_db['mean_s_izq']
data['stdev_bn'] = models_db['stdev_bn_der'] - models_db['stdev_bn_izq']
data['stdev_h'] = models_db['stdev_h_der'] - models_db['stdev_h_izq']
data['stdev_l'] = models_db['stdev_l_der'] - models_db['stdev_l_izq']
data['stdev_s'] = models_db['stdev_s_der'] - models_db['stdev_s_izq']

data['OD_bench'] = models_db['od_bench_der'] - models_db['od_bench_izq']
data['OD_bicycle_der'] = models_db['od_bicycle_der']
data['OD_bicycle_izq'] = models_db['od_bicycle_izq']
data['OD_boat'] = models_db['od_boat_der'] - models_db['od_boat_izq']
data['OD_bus_der'] = models_db['od_bus_der'] 
data['OD_bus_izq'] = models_db['od_bus_izq']
data['OD_car_der'] = models_db['od_car_der'] 
data['OD_car_izq'] = models_db['od_car_izq']
data['OD_chair'] = models_db['od_chair_der'] - models_db['od_chair_izq']
data['OD_fire_hydrant'] = models_db['od_fire_hydrant_der'] - models_db['od_fire_hydrant_izq']
data['OD_motorcycle_der'] = models_db['od_motorcycle_der']
data['OD_motorcycle_izq'] = models_db['od_motorcycle_izq']
data['OD_person_der'] = models_db['od_person_der']
data['OD_person_izq'] = models_db['od_person_izq']
data['OD_potted_plant'] = models_db['od_potted_plant_der'] - models_db['od_potted_plant_izq']
data['OD_stop_sign'] = models_db['od_stop_sign_der'] - models_db['od_stop_sign_izq']
data['OD_traffic_light_der'] = models_db['od_traffic_light_der'] 
data['OD_traffic_light_izq'] = models_db['od_traffic_light_izq']
data['OD_train'] = models_db['od_train_der'] - models_db['od_train_izq']
data['OD_truck_der'] = models_db['od_truck_der'] 
data['OD_truck_izq'] = models_db['od_truck_izq']
data['OD_umbrella'] = models_db['od_umbrella_der'] - models_db['od_umbrella_izq']

data = data.fillna(0)

In [21]:
data.shape
data.describe().T

,count,mean,std,min,25%,50%,75%,max
N_Bicyclist,26913.0,0.000018,0.010008,-0.154173,0.000000,0.000000,0.000000,0.147956
S_Bicyclist,26913.0,-0.000193,0.029867,-0.194216,-0.009982,0.000000,0.009134,0.194216
N_Building,26913.0,0.000560,0.178954,-0.453965,-0.127997,0.000000,0.129056,0.457195
S_Building,26913.0,-0.001079,0.134654,-0.425563,-0.094957,-0.001802,0.092811,0.410521
N_Car,26913.0,0.000088,0.079290,-0.342673,-0.034908,0.000000,0.035548,0.360941
S_Car,26913.0,-0.000510,0.137146,-0.384713,-0.097614,0.000143,0.096518,0.380547
N_Fence,26913.0,0.000643,0.111108,-0.371077,-0.073844,0.000000,0.077032,0.365135
S_Fence,26913.0,0.000361,0.107393,-0.376290,-0.069536,0.000129,0.071111,0.377582
N_Pavement,26913.0,0.000364,0.061185,-0.427551,-0.005447,0.000000,0.006357,0.424781
S_Pavement,26913.0,0.000061,0.087209,-0.362749,-0.054776,-0.000598,0.054219,0.348267


In [22]:
data.head()

,N_Bicyclist,S_Bicyclist,N_Building,S_Building,N_Car,S_Car,N_Fence,S_Fence,N_Pavement,S_Pavement,...,OD_person_der,OD_person_izq,OD_potted_plant,OD_stop_sign,OD_traffic_light_der,OD_traffic_light_izq,OD_train,OD_truck_der,OD_truck_izq,OD_umbrella
1,0.000000,0.000000,-0.046391,0.047152,-0.188574,-0.043097,-0.205750,0.162763,-0.012483,-0.037784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.012586,0.047114,-0.126638,0.002775,-0.053278,0.097995,0.012680,0.001044,-0.047849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,0.000000,0.000000,-0.179175,-0.180889,-0.003552,-0.145841,0.000000,0.153168,0.000000,-0.062994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.004671,-0.022797,-0.318003,-0.107100,-0.062911,0.021984,0.022517,0.101672,-0.052015,-0.084232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.000000,0.013914,0.174592,0.056156,0.011760,0.157688,0.178200,-0.064230,0.006546,-0.046788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
all_data = users.join(data)

In [24]:
all_data.loc[(all_data['question']==1),'OD_car_der'].describe().T

count    7709.000000
mean        2.936568
std         3.673846
min         0.000000
25%         0.000000
50%         2.000000
75%         4.000000
max        37.000000
Name: OD_car_der, dtype: float64

In [25]:
# all_data.shape
all_data.head()

,question,choice,user,age,gender,education,transport,N_Bicyclist,S_Bicyclist,N_Building,...,OD_person_der,OD_person_izq,OD_potted_plant,OD_stop_sign,OD_traffic_light_der,OD_traffic_light_izq,OD_train,OD_truck_der,OD_truck_izq,OD_umbrella
1,3,1,1,24,0,5,3,0.000000,0.000000,-0.046391,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,24,0,5,3,0.000000,0.012586,0.047114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,2,1,821,38,1,5,3,0.000000,0.000000,-0.179175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,2,1,24,0,5,3,-0.004671,-0.022797,-0.318003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,3,2,1,24,0,5,3,0.000000,0.013914,0.174592,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
all_data.to_csv('./../00 Biogeme Input/log_data-sin_filtrar.dat', sep='\t', header=True, index=False, doublequote=True, decimal='.')